In [ ]:
!pip install  ultralytics

##Prepare

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import torch
# Check if CUDA is available, else fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

##First Training

In [ ]:
# Load the pre-trained YOLOv8 model
model = YOLO("yolov8n-seg.pt")
results = model.train(data=".../config.yaml",
                      epochs=2,
                      imgsz=640,
                      batch=64,
                      device=device,
                      project=".../runs")
del model
torch.cuda.empty_cache()

##Test Model

In [ ]:
# Paths
modelPath = '.../runs/medium50ep/weights/last.pt'
testPath = '.../testImg.jpg'

# Load image
img = cv2.imread(testPath)

# Resize the image to 640x640
img_resized = cv2.resize(img, (640, 640))

# Normalize the image (convert pixel values to the range [0, 1])
img_resized = img_resized / 255.0

# Convert the image to a tensor and move it to the device
img_tensor = torch.tensor(img_resized).permute(2, 0, 1).unsqueeze(0).float().to(device)  # Shape becomes (1, 3, 640, 640)

# Load model and move it to the MPS device
model1 = YOLO(modelPath)
model1.to(device)  # Move the model to MPS

# Run inference
results = model1(img_tensor)

# Visualize and save the image with bounding boxes and segmentation masks
annotated_image = results[0].plot()  # Plots the bounding boxes and masks on the image

# Save the annotated image
cv2.imwrite('.../output_annotated.png', annotated_image)

# Optionally display the image using Matplotlib
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()